In [178]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [179]:
##paths
root='/content/drive/MyDrive/BTP PROJECT/'
data_path=root+'data/AmalREC_DBPEDIA/'
dataset_path=data_path+'dataset/'
test_df_path=dataset_path+'final_test_set_fused_gemini.csv'
train_df_path=dataset_path+'final_train_set_fused_gemini.csv'
relations_file_path=data_path+'relations.txt'
### this files will be created after u run the codes  so make sure to kept folder paths correctly
null_train_path=data_path+'train_na_rows.csv'
null_test_path=data_path+'test_na_rows.csv'
chinese_train_path=data_path+'chinese_sentences_train.csv'
chinese_test_path=data_path+'chinese_sentences_test.csv'
slot_dict_path=data_path+'slot.pt'
label_dict_path=data_path+'value_dict.pt'
label_dict_formatted_path=data_path+'value_dict_formatted.pt'
##
seed=42 ## for partition of train set into train and dev
dev_size=0.10
### experiment folder to save label dict value dict and test train dev data splits

exp_folder='exp2_amalrec_hpt/'
exp_folder_path=root+exp_folder
data_split_folder_path=exp_folder_path+'datasplits/'
train_path=data_split_folder_path+'amalrec_train.jsonl'
dev_path=data_split_folder_path+'amalrec_dev.jsonl'
test_path=data_split_folder_path+'amalrec_test.jsonl'



In [180]:
import pandas as pd
import numpy as np

In [181]:
def extract_sentence(prompt, target_sentence_number):
    lines = prompt.strip().split('\n')
    target_sentence = None
    for line in lines:
        if line.startswith(f'Sentence{target_sentence_number}:'):
            target_sentence = line.split(': ', 1)[1]
            break
    return target_sentence

In [182]:
train_df=pd.read_csv(train_df_path)
test_df=pd.read_csv(test_df_path)

In [183]:
print(test_df.columns,train_df.columns)

Index(['E1', 'E1_TYPE', 'RELATION', 'E2', 'E2_TYPE', 'ranker',
       'gemini_fused_Sentences', 'fusion_prompt'],
      dtype='object') Index(['E1', 'E1_TYPE', 'RELATION', 'E2', 'E2_TYPE', 'gemini_fused_Sentences',
       'fusion_prompt'],
      dtype='object')


#### We are intended to extract the ECB sentence from the fusion prompt

In [184]:
row_index = 0  # Change this to the desired row index
print(extract_sentence(train_df.loc[row_index, 'fusion_prompt'],5))

Nikolassee is located in the administrative district of Steglitz-Zehlendorf, which is located in the city of Berlin.


In [185]:
null_counts_train = train_df.isnull().sum()
null_counts_test= test_df.isnull().sum()
print("Null values in train DataFrame:")
for column, count in null_counts_train.items():
    print(f"{column}: {count}")
print("\nNull values in test DataFrame:")
for column, count in null_counts_test.items():
    print(f"{column}: {count}")

Null values in train DataFrame:
E1: 0
E1_TYPE: 0
RELATION: 0
E2: 0
E2_TYPE: 0
gemini_fused_Sentences: 56
fusion_prompt: 56

Null values in test DataFrame:
E1: 0
E1_TYPE: 0
RELATION: 0
E2: 0
E2_TYPE: 0
ranker: 0
gemini_fused_Sentences: 12
fusion_prompt: 12


In [186]:
common_columns = train_df.columns.intersection(test_df.columns)
train_df = train_df[common_columns]
test_df = test_df[common_columns]

#### We are checking for matching duplicates by combining test train

In [187]:
#just to check the common rows // duplicates
combined_df = pd.concat([train_df, test_df], ignore_index=True)
duplicate_count = combined_df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

Number of duplicate rows: 0


In [188]:
test_na_df = test_df[test_df.isnull().any(axis=1)]
train_na_df = train_df[train_df.isnull().any(axis=1)]
test_na_df.to_csv(null_train_path,index=False)
train_na_df.to_csv(null_test_path, index=False)
print("Files 'test_na_rows.csv' and 'train_na_rows.csv' have been created with rows containing NaNs.")

Files 'test_na_rows.csv' and 'train_na_rows.csv' have been created with rows containing NaNs.


### Dataset characterstic:
    some sentences contains "__1" and "__2"
    some other sentences contains "_" as a part of cannonicalization
    Some Sentences consists of NonASCII-Characters such as chinese
    Consists of Null Sentences
    Regarding other noises there is research on observations is currently going on

### EXP2-3:
    We will consider to remove NULL and chinese characters as for we are using
    bert-base-uncased as our pretrained LLM it's not designed for chinese characters
    and the they are also minor as per the following study


In [189]:
# sentence_with_double_underscore = test_null_filtered['gemini_fused_Sentences'].str.contains('__').sum()
# # E1_with_2=train_df['E1'].str.contains('__').sum()
# # E2_with_1 = train_df['E2'].str.contains('__').sum()
# # E2_with_2=train_df['E2'].str.contains('__2').sum()
# verify_train_double = test_null_filtered[test_null_filtered['gemini_fused_Sentences'].str.contains('__')]

# verify_train_double.to_csv('double_underscore_rows_in_train.csv', index=False)

In [190]:
# sentence_with_double_underscore = test_null_filtered['gemini_fused_Sentences'].str.contains('__').sum()
# print(sentence_with_double_underscore)

In [191]:
# sentence_with_double_underscore = train_null_filtered['gemini_fused_Sentences'].str.contains('__').sum()
# print(sentence_with_double_underscore)

In [192]:
# E1_with_1 = test_df['E1'].str.endswith('__1').sum()
# E1_with_2=test_df['E1'].str.endswith('__2').sum()
# E2_with_1 = test_df['E2'].str.endswith('__1').sum()
# E2_with_2=test_df['E2'].str.endswith('__2').sum()
# print(E1_with_1, E1_with_2)
# print(E2_with_1, E2_with_2)

In [193]:
# filtered_E2_train = train_df[train_df['E2'].str.endswith('__1') | train_df['E2'].str.endswith('__2')]
# filtered_E1_train = train_df[train_df['E1'].str.endswith('__1') | train_df['E1'].str.endswith('__2')]
# filtered_E2_test = test_df[test_df['E2'].str.endswith('__1') | test_df['E2'].str.endswith('__2')]
# filtered_E1_test = test_df[test_df['E1'].str.endswith('__1') | test_df['E1'].str.endswith('__2')]

# filtered_rows_train = pd.concat([filtered_E2_train, filtered_E1_train]).drop_duplicates()
# filtered_rows_train.to_csv('double_underscore_rows_in_train.csv', index=False)
# filtered_rows_test = pd.concat([filtered_E2_test, filtered_E1_test]).drop_duplicates()
# filtered_rows_test.to_csv('double_underscore_rows_in_test.csv', index=False)

# print("Filtered rows saved to those files")

In [194]:
test_df = test_df.dropna()
train_df = train_df.dropna()
print(f"Shape of combined DataFrame after removing nulls: {train_df.shape}")
print(f"Shape of combined DataFrame after removing nulls: {test_df.shape}")

Shape of combined DataFrame after removing nulls: (168451, 7)
Shape of combined DataFrame after removing nulls: (15035, 7)


#### function to identify sentences contain chinese characters

In [195]:
import re
def contains_chinese(text):
    # The Unicode range for Chinese characters is \u4e00 to \u9fff
    return bool(re.search(r'[\u4e00-\u9fff]', text))


In [196]:
chinese_sentences_test = test_df[test_df['gemini_fused_Sentences'].apply(contains_chinese)]
print("Sentences containing Chinese characters:")
for _, row in chinese_sentences_test.iterrows():
    print(row['gemini_fused_Sentences'], row['E1'], row['E2'])
print(len(chinese_sentences_test))

Sentences containing Chinese characters:
While the Monte Amiata, located in the Amiata region of Tuscany, is situated in the scenic country of Italy, the province of Piedra, where it resides, lies in its南部. Monte_Amiata Italy
The Gala Television Corporation's television channel, GTV Variety Show (Chinese: 八大綜合台), is headquartered in the beautiful country of Taiwan. GTV_Variety_Show Taiwan
Michael Caine's婚姻关系是通过与妻子Shakira Caine的牢固紐帶維繫在一起的，並建立於寵愛與相互尊重之上。 Michael_Caine Shakira_Caine
Quebec Autoroute 610 traverses Sherbrooke city, connecting the vibrant urban center with neighboring郊区. Quebec_Autoroute_610 Sherbrooke
The起点of Pennsylvania Route 372, located in Lower Chanceford Township within York County, Pennsylvania, serves as a key transportation route. Pennsylvania_Route_372 Lower_Chanceford_Township,_York_County,_Pennsylvania
Louisiana State Route 4, a vital transportation thoroughfare, has its起点 in Shreveport, Louisiana, a city renowned for its vibrant culture and historical significa

In [197]:
chinese_sentences_train = train_df[train_df['gemini_fused_Sentences'].apply(contains_chinese)]
print("Sentences containing Chinese characters:")
for _, row in chinese_sentences_train.iterrows():
    print(row['gemini_fused_Sentences'], row['E1'], row['E2'])
print(len(chinese_sentences_train))

Sentences containing Chinese characters:
Cheyenne, Wyoming, serves as the起点 of Wyoming Highway 221, a vital transportation corridor for commuters and businesses. Wyoming_Highway_221 Cheyenne,_Wyoming
The起点 of Manitoba Highway 20A, a significant roadway in the Canadian province of Manitoba, is the lively city of Dauphin, which hosts the Dauphin Countryfest music festival every year. Manitoba_Highway_20A Dauphin,_Manitoba
The Cité Mer et Soleil Station, a cornerstone of Algiers' public transportation infrastructure, is efficiently operated by Algiers Metro, a longstanding and reliable organization dedicated to providing便捷 and convenient rail services to the city of Algiers. Cité_Mer_et_Soleil_Station Algiers_Metro
China, the location of CCTV-证券资讯's broadcast, is a significant market for the organization's financial news and information due to its large consumer base. CCTV-证券资讯 China
In Cedar Bluff, Virginia, the radio station WYRV, which broadcasts a variety of formats, serves as a vital

In [198]:
chinese_sentences_train.to_csv(chinese_train_path, index=False)
chinese_sentences_test.to_csv(chinese_test_path, index=False)


In [199]:
test_df = test_df[~test_df['gemini_fused_Sentences'].apply(contains_chinese)]
train_df = train_df[~train_df['gemini_fused_Sentences'].apply(contains_chinese)]

In [200]:
train_df['ECB'] = train_df['fusion_prompt'].apply(lambda x: extract_sentence(x, 5))
test_df['ECB'] =test_df['fusion_prompt'].apply(lambda x: extract_sentence(x, 5))

In [201]:
test_df.head(3)

,E1,E1_TYPE,RELATION,E2,E2_TYPE,gemini_fused_Sentences,fusion_prompt,ECB
0,Roman_Polom,Person,team,Czech_Republic_national_under-18_football_team,Organization,Roman Polom recently joined the coaching staff...,"\nGiven the entity, entity type and the relati...",Roman Polom is the head coach of the Czech Rep...
1,Abdoulay_Diaby,Person,team,Lille_OSC,Organization,Lille Olympique Sporting Club's former player ...,"\nGiven the entity, entity type and the relati...","Abdoulay Diaby, a professional footballer who ..."
2,Laurent_Mohellebi,Person,team,AS_Monaco_FC,Organization,"Laurent Mohellebi, a French midfielder, gained...","\nGiven the entity, entity type and the relati...","Laurent Mohellebi, a French footballer, was a ..."


In [202]:
req_cols = ['E1', 'E1_TYPE', 'RELATION', 'E2', 'E2_TYPE', 'ECB', 'gemini_fused_Sentences']
test_df =test_df[req_cols]
train_df =train_df[req_cols]

In [203]:
entity_types=["Person","Location","Organization"]
associations=[]
for x in entity_types:
    for y in entity_types:
        associations.append(f"{x}-{y}")
print(associations)

['Person-Person', 'Person-Location', 'Person-Organization', 'Location-Person', 'Location-Location', 'Location-Organization', 'Organization-Person', 'Organization-Location', 'Organization-Organization']


In [204]:

def create_mapping(file_path):
    mapping = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
    current_root = None
    for line in lines:

        if not line:
            continue
        line = line.strip()
        if line:
            for x in associations:
                if line.startswith(x):
                    current_root = line
                    break
            else:
                if current_root:
                    labels=line.split('/')
                    mapping[f"{current_root}_{labels[-1]}"] = f"Root/{current_root}/{line}"

    return mapping

In [205]:
mapping = create_mapping(relations_file_path)

In [206]:
train_df['tuple'] = train_df['E1_TYPE'] + '-' + train_df['E2_TYPE'] + '_' + train_df['RELATION']
test_df['tuple'] = test_df['E1_TYPE'] + '-' + test_df['E2_TYPE'] + '_' + test_df['RELATION']

In [207]:
train_df.head(3)

,E1,E1_TYPE,RELATION,E2,E2_TYPE,ECB,gemini_fused_Sentences,tuple
0,Nikolassee,Location,administrativeDistrict,Steglitz-Zehlendorf,Location,Nikolassee is located in the administrative di...,As a part of the Steglitz-Zehlendorf administr...,Location-Location_administrativeDistrict
1,Köpenick,Location,administrativeDistrict,Treptow-Köpenick,Location,Köpenick (German pronunciation: [ˈkøːpənɪk] ()...,"Köpenick, a locality in Berlin, is situated at...",Location-Location_administrativeDistrict
2,Othmarschen,Location,administrativeDistrict,"Altona,_Hamburg",Location,Othmarschen is a quarter in the Altona borough...,"As an administrative district, Othmarschen is ...",Location-Location_administrativeDistrict


In [208]:
all_tuples_set = set(train_df['tuple']).union(set(test_df['tuple']))
print("Collected tuples set:")
print(all_tuples_set)

Collected tuples set:
{'Location-Location_routeEnd', 'Person-Organization_awardedBy', 'Person-Location_significantProject', 'Person-Organization_memberOfPoliticalParty', 'Organization-Person_founder', 'Location-Location_mouthRegion', 'Organization-Person_president', 'Organization-Organization_recordLabel', 'Person-Organization_recordLabel', 'Location-Organization_managedBy', 'Location-Location_sourceRegion', 'Organization-Organization_politicalPartyInLegislature', 'Location-Location_largestCity', 'Organization-Location_city', 'Location-Location_leftTributary', 'Person-Location_placeOfDeath', 'Location-Location_river', 'Person-Person_inOfficeVicePresident', 'Organization-Location_localAuthority', 'Location-Location_province', 'Organization-Person_chairperson', 'Organization-Organization_internationalAffiliation', 'Person-Location_hometown', 'Location-Location_lakeOutflow', 'Location-Location_canton', 'Location-Organization_architect', 'Organization-Person_secretaryGeneral', 'Organizatio

In [209]:
len(set(train_df['tuple'])),len(set(test_df['tuple']))

(227, 255)

In [210]:
valid_relations=set(mapping.keys())

In [211]:
print(len(all_tuples_set),len(valid_relations))

255 281


In [212]:
extra_in_df = all_tuples_set - valid_relations
extra_in_list = valid_relations - all_tuples_set
print("extra_relations which are not in dbpedia_heirarchy_list:", len(extra_in_df),"\n","extra_relations which are not in combined_df",len(extra_in_list))

extra_relations which are not in dbpedia_heirarchy_list: 0 
 extra_relations which are not in combined_df 26


In [213]:
print(extra_in_list)

{'Location-Location_routeLocation', 'Organization-Person_member', 'Person-Organization_memberOf', 'Organization-Person_associatedWith', 'Organization-Organization_company', 'Organization-Person_people', 'Organization-Organization_organization', 'Organization-Organization_associatedWith', 'Organization-Organization_politicalParty', 'Person-Person_advisor', 'Location-Location_mountain', 'Person-Person_student', 'Person-Location_locatedAt', 'Person-Location_state', 'Person-Organization_coachOf', 'Organization-Person_officeholder', 'Location-Location_area', 'Person-Organization_formerCoachOf', 'Location-Location_administrativeDivision', 'Organization-Organization_legislativeBody', 'Location-Person_officeholder', 'Organization-Location_stadium', 'Location-Location_tributary', 'Person-Organization_club', 'Location-Location_mouthOfWaterCourse', 'Person-Person_officeholder'}


In [214]:
test_df['relation_label'] = test_df['tuple'].replace(mapping)
train_df['relation_label'] = train_df['tuple'].replace(mapping)

In [215]:
train_df.head(3)

,E1,E1_TYPE,RELATION,E2,E2_TYPE,ECB,gemini_fused_Sentences,tuple,relation_label
0,Nikolassee,Location,administrativeDistrict,Steglitz-Zehlendorf,Location,Nikolassee is located in the administrative di...,As a part of the Steglitz-Zehlendorf administr...,Location-Location_administrativeDistrict,Root/Location-Location/administrativeDivision/...
1,Köpenick,Location,administrativeDistrict,Treptow-Köpenick,Location,Köpenick (German pronunciation: [ˈkøːpənɪk] ()...,"Köpenick, a locality in Berlin, is situated at...",Location-Location_administrativeDistrict,Root/Location-Location/administrativeDivision/...
2,Othmarschen,Location,administrativeDistrict,"Altona,_Hamburg",Location,Othmarschen is a quarter in the Altona borough...,"As an administrative district, Othmarschen is ...",Location-Location_administrativeDistrict,Root/Location-Location/administrativeDivision/...


In [216]:
columns_to_keep = ['E1', 'E1_TYPE', 'RELATION', 'E2', 'E2_TYPE', 'ECB', 'gemini_fused_Sentences', 'relation_label']
test_df = test_df[columns_to_keep]
train_df = train_df[columns_to_keep]


In [217]:
def get_labels(label):
    sample_label=[]
    heir_list=label.split('/')
    for l in range(2,len(heir_list)+1):
        label='/'.join(heir_list[1:l])
        if label not in sample_label and not '':
            sample_label.append(label)
    return sample_label



In [218]:
test_df['sample_labels']=test_df['relation_label'].apply(get_labels)
train_df['sample_labels']=train_df['relation_label'].apply(get_labels)
train_df.head(3)

,E1,E1_TYPE,RELATION,E2,E2_TYPE,ECB,gemini_fused_Sentences,relation_label,sample_labels
0,Nikolassee,Location,administrativeDistrict,Steglitz-Zehlendorf,Location,Nikolassee is located in the administrative di...,As a part of the Steglitz-Zehlendorf administr...,Root/Location-Location/administrativeDivision/...,"[Location-Location, Location-Location/administ..."
1,Köpenick,Location,administrativeDistrict,Treptow-Köpenick,Location,Köpenick (German pronunciation: [ˈkøːpənɪk] ()...,"Köpenick, a locality in Berlin, is situated at...",Root/Location-Location/administrativeDivision/...,"[Location-Location, Location-Location/administ..."
2,Othmarschen,Location,administrativeDistrict,"Altona,_Hamburg",Location,Othmarschen is a quarter in the Altona borough...,"As an administrative district, Othmarschen is ...",Root/Location-Location/administrativeDivision/...,"[Location-Location, Location-Location/administ..."


In [219]:

from collections import defaultdict
label_dict={}
hiera = defaultdict(set)
rev_dict={}

In [220]:
train_labels=train_df['sample_labels'].tolist()
test_labels=test_df['sample_labels'].tolist()
labels=train_labels+test_labels

In [221]:
for l in labels:
    for l_ in l:
        split=l_.split('/')
        if l_ not in label_dict:
            label_dict[l_]=len(label_dict)
        for i in range(1,len(split)):
            hiera[label_dict['/'.join(split[:i])]].add(label_dict['/'.join(split[:i + 1])])
            rev_dict['/'.join(split[:i +1])] = '/'.join(split[:i])


In [222]:
value_dict = {i: v.split('/')[-1] for v, i in
                  label_dict.items()}

In [223]:
print(hiera)

defaultdict(<class 'set'>, {0: {1, 4, 14, 18, 19, 22, 28, 29, 30, 31, 32, 33, 34, 38, 42, 43, 44, 46, 54, 58, 67, 68, 70, 72, 77, 78, 85}, 1: {2, 40, 9, 13, 16, 26}, 2: {3}, 5: {37, 6, 11, 47, 49, 84, 27, 62, 63}, 7: {64, 36, 8, 12, 76, 50, 20, 21, 24, 25}, 9: {10}, 14: {88, 35, 60, 15}, 16: {17}, 22: {23}, 38: {69, 39}, 40: {41, 66}, 44: {65, 53, 45}, 31: {48}, 50: {51}, 26: {52}, 54: {56, 57, 71, 55}, 58: {59, 61}, 72: {73, 74, 75}, 78: {80, 81, 82, 79}, 68: {83}, 62: {86}, 18: {87}, 89: {259, 132, 135, 265, 267, 268, 142, 144, 146, 147, 276, 152, 153, 159, 162, 164, 90, 96, 97, 103, 109, 113, 119, 121, 122}, 91: {129, 101, 102, 261, 104, 136, 263, 266, 141, 111, 124, 143, 271, 115, 116, 92, 126, 127}, 93: {128, 99, 158, 105, 137, 107, 139, 112, 145, 148, 277, 118, 279, 154, 123, 94}, 90: {150, 270, 134, 95}, 97: {98, 140, 269}, 99: {100}, 105: {163, 106, 108, 272, 114, 275, 117}, 109: {125, 110, 157}, 119: {120, 155, 156}, 127: {130, 131}, 132: {133}, 137: {138}, 148: {149}, 114: {2

In [224]:
#torch to save for HPT
import torch
torch.save(value_dict, label_dict_path)
torch.save(hiera, slot_dict_path)

In [225]:
import re
def format_string(value):
    if '-' in value:
        return value.lower()
    else:
        value = re.sub(r'(?<!^)(?=[A-Z])', ' ', value)
        value = ' '.join(value.split()).lower()
        return value

formatted_dict = {k: format_string(v) for k, v in value_dict.items()}
print(formatted_dict)

{0: 'location-location', 1: 'administrative division', 2: 'district', 3: 'administrative district', 4: 'archipelago', 5: 'location-organization', 6: 'architect', 7: 'location-person', 8: 'architect', 9: 'department', 10: 'arrondissement', 11: 'builder', 12: 'builder', 13: 'canton', 14: 'city', 15: 'capital city', 16: 'county', 17: 'ceremonial county', 18: 'country', 19: 'crosses', 20: 'designer', 21: 'discoverer', 22: 'state', 23: 'federal state', 24: 'first ascent person', 25: 'founder', 26: 'frazioni', 27: 'governing body', 28: 'has junction with', 29: 'highest place', 30: 'is part of', 31: 'island', 32: 'lake', 33: 'lake inflow', 34: 'lake outflow', 35: 'largest city', 36: 'leader', 37: 'leader party', 38: 'tributary', 39: 'left tributary', 40: 'area', 41: 'lieutenancy area', 42: 'located at', 43: 'located in area', 44: 'mountain', 45: 'lowest mountain', 46: 'lowest place', 47: 'maintained by', 48: 'major island', 49: 'managed by', 50: 'officeholder', 51: 'mayor', 52: 'metropolitan 

In [226]:
torch.save(formatted_dict, label_dict_formatted_path)

In [227]:
label_dict

{'Location-Location': 0,
 'Location-Location/administrativeDivision': 1,
 'Location-Location/administrativeDivision/district': 2,
 'Location-Location/administrativeDivision/district/administrativeDistrict': 3,
 'Location-Location/archipelago': 4,
 'Location-Organization': 5,
 'Location-Organization/architect': 6,
 'Location-Person': 7,
 'Location-Person/architect': 8,
 'Location-Location/administrativeDivision/department': 9,
 'Location-Location/administrativeDivision/department/arrondissement': 10,
 'Location-Organization/builder': 11,
 'Location-Person/builder': 12,
 'Location-Location/administrativeDivision/canton': 13,
 'Location-Location/city': 14,
 'Location-Location/city/capitalCity': 15,
 'Location-Location/administrativeDivision/county': 16,
 'Location-Location/administrativeDivision/county/ceremonialCounty': 17,
 'Location-Location/country': 18,
 'Location-Location/crosses': 19,
 'Location-Person/designer': 20,
 'Location-Person/discoverer': 21,
 'Location-Location/state': 22

In [228]:
def get_label_ids(value):
    return [label_dict[i] for i in value]
train_df['label_ids']=train_df['sample_labels'].apply(get_label_ids)
test_df['label_ids']=test_df['sample_labels'].apply(get_label_ids)
test_df.head()

,E1,E1_TYPE,RELATION,E2,E2_TYPE,ECB,gemini_fused_Sentences,relation_label,sample_labels,label_ids
0,Roman_Polom,Person,team,Czech_Republic_national_under-18_football_team,Organization,Roman Polom is the head coach of the Czech Rep...,Roman Polom recently joined the coaching staff...,Root/Person-Organization/team,"[Person-Organization, Person-Organization/team]","[168, 191]"
1,Abdoulay_Diaby,Person,team,Lille_OSC,Organization,"Abdoulay Diaby, a professional footballer who ...",Lille Olympique Sporting Club's former player ...,Root/Person-Organization/team,"[Person-Organization, Person-Organization/team]","[168, 191]"
2,Laurent_Mohellebi,Person,team,AS_Monaco_FC,Organization,"Laurent Mohellebi, a French footballer, was a ...","Laurent Mohellebi, a French midfielder, gained...",Root/Person-Organization/team,"[Person-Organization, Person-Organization/team]","[168, 191]"
3,Bob_Shankly,Person,team,Alloa_Athletic_F.C.,Organization,"Bob Shankly, a professional football player an...","Bob Shankly, a former professional football pl...",Root/Person-Organization/team,"[Person-Organization, Person-Organization/team]","[168, 191]"
4,Byron_Webster,Person,team,Harrogate_Town_F.C.,Organization,Harrogate Town F.C. is the club that Byron Web...,"Byron Webster, a prominent player, has been in...",Root/Person-Organization/team,"[Person-Organization, Person-Organization/team]","[168, 191]"


In [229]:
train_df.head()

,E1,E1_TYPE,RELATION,E2,E2_TYPE,ECB,gemini_fused_Sentences,relation_label,sample_labels,label_ids
0,Nikolassee,Location,administrativeDistrict,Steglitz-Zehlendorf,Location,Nikolassee is located in the administrative di...,As a part of the Steglitz-Zehlendorf administr...,Root/Location-Location/administrativeDivision/...,"[Location-Location, Location-Location/administ...","[0, 1, 2, 3]"
1,Köpenick,Location,administrativeDistrict,Treptow-Köpenick,Location,Köpenick (German pronunciation: [ˈkøːpənɪk] ()...,"Köpenick, a locality in Berlin, is situated at...",Root/Location-Location/administrativeDivision/...,"[Location-Location, Location-Location/administ...","[0, 1, 2, 3]"
2,Othmarschen,Location,administrativeDistrict,"Altona,_Hamburg",Location,Othmarschen is a quarter in the Altona borough...,"As an administrative district, Othmarschen is ...",Root/Location-Location/administrativeDivision/...,"[Location-Location, Location-Location/administ...","[0, 1, 2, 3]"
3,Blankenese,Location,administrativeDistrict,Hamburg,Location,Blankenese is a suburban quarter in the admini...,"Blankenese's administrative district, Hamburg,...",Root/Location-Location/administrativeDivision/...,"[Location-Location, Location-Location/administ...","[0, 1, 2, 3]"
4,Tempelhof,Location,administrativeDistrict,Tempelhof-Schöneberg,Location,Tempelhof is a locality of Berlin within the a...,"Tempelhof-Schöneberg, which encompasses the lo...",Root/Location-Location/administrativeDivision/...,"[Location-Location, Location-Location/administ...","[0, 1, 2, 3]"


In [230]:
train_df['Suffix'] = ('What is the relation between "'+ train_df['E1'] +'" and "'+ train_df['E2'] +'"? The answer is ')
test_df['Suffix'] = ('What is the relation between "'+ test_df['E1'] +'" and "'+ test_df['E2'] +'"? The answer is ')
train_df.head()

,E1,E1_TYPE,RELATION,E2,E2_TYPE,ECB,gemini_fused_Sentences,relation_label,sample_labels,label_ids,Suffix
0,Nikolassee,Location,administrativeDistrict,Steglitz-Zehlendorf,Location,Nikolassee is located in the administrative di...,As a part of the Steglitz-Zehlendorf administr...,Root/Location-Location/administrativeDivision/...,"[Location-Location, Location-Location/administ...","[0, 1, 2, 3]","What is the relation between ""Nikolassee"" and ..."
1,Köpenick,Location,administrativeDistrict,Treptow-Köpenick,Location,Köpenick (German pronunciation: [ˈkøːpənɪk] ()...,"Köpenick, a locality in Berlin, is situated at...",Root/Location-Location/administrativeDivision/...,"[Location-Location, Location-Location/administ...","[0, 1, 2, 3]","What is the relation between ""Köpenick"" and ""T..."
2,Othmarschen,Location,administrativeDistrict,"Altona,_Hamburg",Location,Othmarschen is a quarter in the Altona borough...,"As an administrative district, Othmarschen is ...",Root/Location-Location/administrativeDivision/...,"[Location-Location, Location-Location/administ...","[0, 1, 2, 3]","What is the relation between ""Othmarschen"" and..."
3,Blankenese,Location,administrativeDistrict,Hamburg,Location,Blankenese is a suburban quarter in the admini...,"Blankenese's administrative district, Hamburg,...",Root/Location-Location/administrativeDivision/...,"[Location-Location, Location-Location/administ...","[0, 1, 2, 3]","What is the relation between ""Blankenese"" and ..."
4,Tempelhof,Location,administrativeDistrict,Tempelhof-Schöneberg,Location,Tempelhof is a locality of Berlin within the a...,"Tempelhof-Schöneberg, which encompasses the lo...",Root/Location-Location/administrativeDivision/...,"[Location-Location, Location-Location/administ...","[0, 1, 2, 3]","What is the relation between ""Tempelhof"" and ""..."


In [231]:
!pip install unidecode

In [232]:
from unidecode import unidecode
train_df['gemini_fused_Sentences'] = train_df['gemini_fused_Sentences'].str.strip()
train_df['Suffix'] = train_df['Suffix'].str.strip()

# Combine the two columns with a space in between
train_df['token'] = train_df['gemini_fused_Sentences'] + " " + train_df['Suffix']
train_df['token'] = train_df['token'].apply(unidecode)
train_df.head()

,E1,E1_TYPE,RELATION,E2,E2_TYPE,ECB,gemini_fused_Sentences,relation_label,sample_labels,label_ids,Suffix,token
0,Nikolassee,Location,administrativeDistrict,Steglitz-Zehlendorf,Location,Nikolassee is located in the administrative di...,As a part of the Steglitz-Zehlendorf administr...,Root/Location-Location/administrativeDivision/...,"[Location-Location, Location-Location/administ...","[0, 1, 2, 3]","What is the relation between ""Nikolassee"" and ...",As a part of the Steglitz-Zehlendorf administr...
1,Köpenick,Location,administrativeDistrict,Treptow-Köpenick,Location,Köpenick (German pronunciation: [ˈkøːpənɪk] ()...,"Köpenick, a locality in Berlin, is situated at...",Root/Location-Location/administrativeDivision/...,"[Location-Location, Location-Location/administ...","[0, 1, 2, 3]","What is the relation between ""Köpenick"" and ""T...","Kopenick, a locality in Berlin, is situated at..."
2,Othmarschen,Location,administrativeDistrict,"Altona,_Hamburg",Location,Othmarschen is a quarter in the Altona borough...,"As an administrative district, Othmarschen is ...",Root/Location-Location/administrativeDivision/...,"[Location-Location, Location-Location/administ...","[0, 1, 2, 3]","What is the relation between ""Othmarschen"" and...","As an administrative district, Othmarschen is ..."
3,Blankenese,Location,administrativeDistrict,Hamburg,Location,Blankenese is a suburban quarter in the admini...,"Blankenese's administrative district, Hamburg,...",Root/Location-Location/administrativeDivision/...,"[Location-Location, Location-Location/administ...","[0, 1, 2, 3]","What is the relation between ""Blankenese"" and ...","Blankenese's administrative district, Hamburg,..."
4,Tempelhof,Location,administrativeDistrict,Tempelhof-Schöneberg,Location,Tempelhof is a locality of Berlin within the a...,"Tempelhof-Schöneberg, which encompasses the lo...",Root/Location-Location/administrativeDivision/...,"[Location-Location, Location-Location/administ...","[0, 1, 2, 3]","What is the relation between ""Tempelhof"" and ""...","Tempelhof-Schoneberg, which encompasses the lo..."


In [233]:
test_df['gemini_fused_Sentences'] = test_df['gemini_fused_Sentences'].str.strip()
test_df['Suffix'] = test_df['Suffix'].str.strip()

# Combine the two columns with a space in between
test_df['token'] = test_df['gemini_fused_Sentences'] + " " + test_df['Suffix']
test_df['token'] = test_df['token'].apply(unidecode)

## Split the train into 90 train and 10dev to remain consistent with 15k test

In [234]:
from sklearn.model_selection import train_test_split

# Split into train (70%) and temp (30%) without stratification
train_df, dev_df = train_test_split(
    train_df,
    test_size=0.10,
    random_state=seed
)


In [235]:
import json
def write_jsonl(filename, dataset):
    with open(filename, 'w') as f:
        for _, row in dataset.iterrows():
            line = json.dumps({'token': row['token'], 'label': row['label_ids']})
            f.write(line + '\n')

In [236]:


# Save the splits to respective files
write_jsonl(train_path, train_df)
write_jsonl(dev_path, dev_df)
write_jsonl(test_path, test_df)

In [237]:
train_df.shape, dev_df.shape, test_df.shape

((151542, 12), (16838, 12), (15023, 12))

In [244]:

train_relation_labels = set(item for sublist in train_df['label_ids'] for item in sublist)
print(len(train_relation_labels))

259


In [245]:

test_relation_labels = set(item for sublist in test_df['label_ids'] for item in sublist)
print(len(test_relation_labels))

286


In [246]:

dev_relation_labels = set(item for sublist in dev_df['label_ids'] for item in sublist)
print(len(dev_relation_labels))

258


In [247]:
non_train_labels=test_relation_labels-train_relation_labels

In [248]:
for x in non_train_labels:
    print(value_dict[x])

subsidiary
sisterStation
state
formerBandMember
regionServed
formerBroadcastNetwork
successor
targetAirport
rival
splitFromParty
university
europeanAffiliation
region
viceChancellor
secondCommander
schoolPatron
secretaryGeneral
religion
rector
employer
influencedBy
religion
battleFaughtUnder
currentMemberOf
inOfficeVicePrimeMinister
formerHighSchool
nationalTeam


These were never present in the train set in any level

In [253]:
# First flatten the list of label_ids and then count occurrences
label_counts = {}
for ids in train_df['label_ids']:
    for id in ids:
        label_counts[id] = label_counts.get(id, 0) + 1

# Sort by count in descending order
sorted_counts = sorted(label_counts.items(), key=lambda x: x[1])

# Print the counts
for id, count in sorted_counts:
    print(f"Label {formatted_dict[id]} id- {id}: {count} occurrences")

Label coached team id- 192: 1 occurrences
Label chairperson id- 107: 12 occurrences
Label designer id- 20: 21 occurrences
Label academic advisor id- 167: 31 occurrences
Label lowest mountain id- 45: 32 occurrences
Label patron id- 158: 34 occurrences
Label dean id- 117: 36 occurrences
Label alumni id- 94: 36 occurrences
Label is part of id- 135: 52 occurrences
Label lowest place id- 46: 54 occurrences
Label administrative district id- 3: 68 occurrences
Label country for sport id- 196: 72 occurrences
Label agency id- 169: 72 occurrences
Label authority id- 96: 76 occurrences
Label director id- 118: 77 occurrences
Label founder id- 25: 77 occurrences
Label has junction with id- 28: 78 occurrences
Label highest place id- 29: 84 occurrences
Label spouse id- 252: 84 occurrences
Label state id- 253: 87 occurrences
Label state of origin id- 254: 87 occurrences
Label opponent id- 237: 87 occurrences
Label university id- 258: 88 occurrences
Label significant building id- 250: 90 occurrences
Lab

In our 3rd experiment we wont consider the coached team in our test set as it is only occured once in the train data and model wont be learned with just 1 label so it's will be a fair assumption to no consider that class as well along with class which are not present in the train

In [254]:
non_train_labels.add(192)

In [255]:
print(non_train_labels)

{259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 192}


In [257]:
refined_test_df=test_df[~test_df['label_ids'].apply(lambda x: any(item in non_train_labels for item in x))]


In [258]:
refined_test_df.shape

(13541, 12)

FOR EXPERIMENT-3

In [259]:
exp_folder='exp3_amalrec_hpt/'
exp_folder_path=root+exp_folder
data_split_folder_path=exp_folder_path+'datasplits/'
train_path=data_split_folder_path+'amalrec_train.jsonl'
dev_path=data_split_folder_path+'amalrec_dev.jsonl'
test_path=data_split_folder_path+'amalrec_test.jsonl'


In [260]:
# Save the splits to respective files
write_jsonl(train_path, train_df)
write_jsonl(dev_path, dev_df)
write_jsonl(test_path, refined_test_df)

In [261]:

exp_folder='exp2_amalrec_hpt/'
exp_folder_path=root+exp_folder
data_split_folder_path=exp_folder_path+'datasplits/'
test_hidden_path=data_split_folder_path+'amalrec_test_hidden.jsonl'


In [262]:
import json
def write_jsonl_hidden(filename, dataset):
    with open(filename, 'w') as f:
        for _, row in dataset.iterrows():
            line = json.dumps({'token': row['token'], 'label': []})
            f.write(line + '\n')

In [263]:
write_jsonl_hidden(test_hidden_path, test_df)

In [264]:

exp_folder='exp3_amalrec_hpt/'
exp_folder_path=root+exp_folder
data_split_folder_path=exp_folder_path+'datasplits/'
test_hidden_path=data_split_folder_path+'amalrec_test_hidden.jsonl'

In [265]:
write_jsonl_hidden(test_hidden_path, refined_test_df)